In [ ]:
using DataStructures
using Glob
using LargeScaleAnalysis
using ProgressMeter
using PyCall
using PyPlot

In [ ]:
ASNames = pyimport("fetchmesh.asn").ASNames;
asnames = Dict{Int,String}(ASNames.from_url().mapping);

In [ ]:
files = glob("../data/traceroute_v4_1580511600_1580513400_noself_fullmesh/*.processed.json");

In [ ]:
asn_counts = DefaultDict{Int, Set{Tuple{String,String}}}(() -> Set{Tuple{String,String}}())
ixp_counts = DefaultDict{String, Set{Tuple{String,String}}}(() -> Set{Tuple{String,String}}())
all_pairs = Set{Tuple{String,String}}()
@showprogress for file in files
    records = parsefile(Vector{TracerouteRecord}, file)
    for record in records
        pair = (record.from, record.dst_addr)
        push!(all_pairs, pair)
        for hop in record.hops_asn, asn in hop
            push!(asn_counts[asn], pair)
        end
        for hop in record.hops_ix, ix in hop
            push!(ixp_counts[ix], pair)
        end
    end
end

In [ ]:
asn_counts = sort([(k, length(v)) for (k, v) in asn_counts], by = x -> x[2], rev = true);

In [ ]:
ixp_counts = sort([(k, length(v)) for (k, v) in ixp_counts], by = x -> x[2], rev = true);

In [ ]:
[(asn, asnames[asn], count, count / length(all_pairs) * 100) for (asn, count) in asn_counts[1:10]]

In [ ]:
[(ixp, count, count / length(all_pairs) * 100) for (ixp, count) in ixp_counts[1:10]]

In [ ]:
filter(x -> x[2] >= 1000, asn_counts)

In [ ]:
filter(x -> x[2] >= 1000, ixp_counts)